In [ ]:
!pip install git+https://github.com/huggingface/transformers

In [ ]:
!pip install qwen-vl-utils

In [ ]:
!pip install torchvision

In [ ]:
!pip install accelerate

In [3]:
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

In [4]:
if torch.backends.mps.is_available():
    device = "mps"
    print("Using MPS (Metal Performance Shaders) on Mac")
elif torch.cuda.is_available():
    device = "cuda"
    print("Using CUDA (GPU)")
else:
    device = "cpu"
    print("Using CPU")

Using MPS (Metal Performance Shaders) on Mac


In [ ]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)

In [ ]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

In [ ]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)